In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

Importing Twitter Sentimental Dataset

In [2]:
# api to fecth the dataset
!kaggle datasets download -d kazanova/sentiment140

 98% 79.0M/80.9M [00:00<00:00, 148MB/s] 
100% 80.9M/80.9M [00:00<00:00, 123MB/s]


In [3]:
#extracting the comp. file

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset , 'r') as zip:
  zip.extractall()
  print('done..')

done..


Importing Lib


In [16]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer  # Corrected import statement
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
print(stopwords.words('english')) # remove this words from the data cuz this words dont have importance wrt our model.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

In [20]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding  = 'ISO-8859-1')

In [21]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _

In [ ]:
twitter_data.shape

In [23]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


our data is considering first row as col names , so we need to change that!

In [25]:
#naming the column
column_names = ['target' , 'id' , 'date' , 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = column_names ,encoding  = 'ISO-8859-1')

In [26]:
twitter_data.shape

(1600000, 6)

In [28]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [29]:
twitter_data.isna().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [32]:
twitter_data['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

0 -> -ve tweet
1 -> +ve tweet **bold text**

data is not imbalanced so no need to modify it.

In [35]:
twitter_data.replace({'target' : {4:1}}, inplace = True)

In [36]:
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

**Stemming**

stemming is the process of reducing a word to its Root Word
ex: actor , actress , acting == **act**

In [37]:
port_stem = PorterStemmer()


In [41]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-z]', ' ',  content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

***In above funtion***

- first we removed all special char and numbers.
- then lowercase
- split the sentence in orter to check the words for stemming
- checked the words , if word is present in stopwords perform
  join operation below.

In [42]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

Took 50 mins to run💀

In [43]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [46]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [47]:
print(X)
print(Y)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
[0 0 0 ... 1 1 1]


In [50]:
X_train , X_test,  Y_train , Y_test = train_test_split(X , Y , test_size = 0.2 , stratify=Y , random_state=2)

In [51]:
# feature extraction (CONVERTING TEXT TO NUM)
# TfidfVectorizer() will give values to +ve & -ve tweets
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [52]:
print(X_train)

  (0, 464940)	0.4483581441053542
  (0, 247667)	0.4215060595161854
  (0, 116318)	0.3749338694119814
  (0, 194823)	0.5274064910069903
  (0, 372989)	0.3583819096048567
  (0, 458421)	0.27228112326228093
  (1, 169453)	1.0
  (2, 304223)	0.1678648733603879
  (2, 140239)	0.20289994039686138
  (2, 159240)	0.18800868496584808
  (2, 187474)	0.16194060015844067
  (2, 429906)	0.15168139293444055
  (2, 281186)	0.24137202118772233
  (2, 465311)	0.33480686186877023
  (2, 83506)	0.31303964332256906
  (2, 455221)	0.3296073093697297
  (2, 427034)	0.32103058232997417
  (2, 137276)	0.29069581876534506
  (2, 427967)	0.18706371586911208
  (2, 132152)	0.1891875792321668
  (2, 116318)	0.45905705320412793
  (3, 181609)	0.3745702863460754
  (3, 432396)	0.2708400841098632
  (3, 408311)	0.3941744062368762
  (3, 61032)	0.52009780709937
  :	:
  (1279996, 409874)	0.21996814268672968
  (1279996, 455685)	0.27108924963905606
  (1279996, 335154)	0.21190629789318696
  (1279996, 250767)	0.22319402802219876
  (1279996, 3069

In [71]:
print(X_test)

  (0, 442233)	0.17903569516819148
  (0, 429906)	0.3140986667345934
  (0, 419078)	0.34884649404705365
  (0, 408024)	0.21977439030363208
  (0, 294440)	0.17813419553395948
  (0, 285841)	0.4532311769636366
  (0, 180544)	0.2803743470609849
  (0, 146252)	0.23682534421364126
  (0, 140294)	0.25509664355434203
  (0, 112919)	0.367081693610678
  (0, 72943)	0.2678484639307855
  (0, 34872)	0.16236726920078096
  (0, 17977)	0.17180827032540266
  (1, 385085)	0.24302725359248564
  (1, 366102)	0.4820488545932507
  (1, 270721)	0.28432607616832933
  (1, 229396)	0.40557814327518943
  (1, 153709)	0.5722666578553444
  (1, 17977)	0.2084958588978494
  (1, 8801)	0.30376315944222493
  (2, 420861)	0.4330300886388968
  (2, 270795)	0.2562344190631401
  (2, 192882)	0.5885804190746107
  (2, 95488)	0.363168936008175
  (2, 38175)	0.37941511785250515
  :	:
  (319994, 130897)	0.4539824479197656
  (319995, 466849)	0.3209500415545892
  (319995, 442233)	0.22620821621683035
  (319995, 437352)	0.23804373106127552
  (319995, 3

**MODEL BUILDING -: LOGISTIC REGRESSION**

In [54]:
model = LogisticRegression(max_iter=1000) # max_iter will iterate 1000 time on data till it reaches a good accuracy

In [55]:
model.fit(X_train , Y_train)

LogisticRegression(max_iter=1000)

**Model Evaluation**

In [59]:
# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train ,X_train_prediction)

In [60]:
print('Accuracy Score on Training Data : ' , training_data_accuracy)

Accuracy Score on Training Data :  0.811671875


In [61]:
# accuracy score on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test ,X_test_prediction)

In [62]:
print('Accuracy Score on Test Data : ' , test_data_accuracy)

Accuracy Score on Test Data :  0.778546875


**Model Accuracy = 77.8%**

In [63]:
import pickle

Save the model using pickle

In [64]:
filename = 'train_model.sav'
pickle.dump(model , open(filename, 'wb'))

now load the model

In [65]:
loaded_model = pickle.load(open ('/content/train_model.sav', 'rb'))

In [67]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet 😡')

else:
  print('Positive Tweet 😘')

1
[1]
Positive Tweet 😘


In [74]:
X_new = X_test[2]
print(Y_test[2])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet 😡')

else:
  print('Positive Tweet 😘')

0
[0]
Negative Tweet 😡
